In [ ]:
!gdown --id "1Lx6Kxy29Jp5LFtWsszOMUAraTqam2FxG"

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Lx6Kxy29Jp5LFtWsszOMUAraTqam2FxG
To: /content/wav2.zip
100% 421M/421M [00:09<00:00, 42.1MB/s]


In [ ]:
!unzip "/content/wav2.zip"

In [ ]:
import pandas as pd


In [ ]:
import pandas as pd
a = pd.read_csv("/content/test_df.csv")
a

FileNotFoundError: ignored

In [ ]:
df = pd.read_csv("/content/test_df.csv")

In [ ]:
import re

In [ ]:
df["sentence"] = df["sentence"].apply(lambda x: re.sub("[a-zA-Z\<\>（）“”—— !?:;./ ！ 《》：”“!，。]", "", x))

In [ ]:
df

,sentence,path,labels
0,如果你相信这个,/content/wav/000100110.WAV,1
1,使我们从实证的方面去解决生物界的根本问题,/content/wav/000100050.WAV,1
2,又不是英国的消极自由所谓平等,/content/wav/000120092.WAV,1
3,这就是调和,/content/wav/000120328.WAV,1
4,他洋洋得意的据得这种生活狠可以终身了,/content/wav/000110276.WAV,1
...,...,...,...
510,使我们从实证的方面去解决生物界的根本问题,/content/wav/000100142.WAV,0
511,五,/content/wav/000120522.WAV,0
512,葡萄太高了他吃不着,/content/wav/000120269.WAV,0
513,也许隔开两三万里路,/content/wav/000130123.WAV,0


In [ ]:
def pathtowav(id):
  path = f"{id}"
  return path


In [ ]:
import librosa

In [ ]:
def speech_file_to_array_fn(path):
    #bath = pathtowav(a["id"])
    speech_array, sampling_rate = librosa.load(path, sr=16_000)
    #batch["speech"] = speech_array
    #atch["sentence"] = batch["sentence"].upper()
    return speech_array

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import numpy as np

In [ ]:
class Realwav:
  def __init__(self):
    MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-chinese-zh-cn"
    self.processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
    self.model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)
  
  def sound_and_sentence(self,sound,sentence):
    inputs = self.processor(sound, sampling_rate=16_000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = self.model(inputs.input_values, attention_mask=inputs.attention_mask).logits
    
    probs = torch.softmax(logits, dim=-1)
    ref_ids = self.processor(text=sentence)["input_ids"]
    scores = []
    ref_count = 0
    pred_ids = torch.argmax(logits[0], dim=-1)
    for seq_idx in range(pred_ids.shape[0]):
      if pred_ids[seq_idx] != 0:
        print(f"position of the word {sentence[ref_count]}: {seq_idx}")
        ref_id = ref_ids[ref_count]
        conf_score = probs[0, seq_idx, ref_id].tolist()
        scores.append(conf_score)
        print(conf_score)
        ref_count += 1
        if ref_count >= len(ref_ids):
          break
    #print("-" * 100)
    #print(scores)
    sentence_score = np.mean(scores)
    #print(sentence_score)
    return sentence_score, int(sentence_score > 0.5)


        

In [ ]:
rw = Realwav()

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [ ]:
df

,sentence,path,labels
0,如果你相信这个 。,/content/wav/000100110.WAV,1
1,使我们从实证的方面去解决生物界的根本问题 。,/content/wav/000100050.WAV,1
2,又不是英国的消极自由 ， 所谓平等 ，,/content/wav/000120092.WAV,1
3,这就是调和 。,/content/wav/000120328.WAV,1
4,他洋洋得意的据得这种生活狠可以终身了 。,/content/wav/000110276.WAV,1
...,...,...,...
510,使我们从实证的方面去解决生物界的根本问题 。,/content/wav/000100142.WAV,0
511,五,/content/wav/000120522.WAV,0
512,葡萄太高了 ， 他吃不着 ，,/content/wav/000120269.WAV,0
513,也许隔开两三万里路 ，,/content/wav/000130123.WAV,0


In [ ]:
#test class Realwav
sample = df.iloc[0]
id = sample["path"]
sentence = sample["sentence"]
pathwav = pathtowav(id)
filesound = speech_file_to_array_fn(pathwav)
output = rw.sound_and_sentence(filesound,sentence)


position of the word 如: 29
0.9994091987609863
position of the word 果: 45
0.9841655492782593
position of the word 你: 59
0.9850018620491028
position of the word 相: 73
0.9347127676010132
position of the word 信: 86
0.049125611782073975
position of the word 这: 101
0.9993152618408203
position of the word 个: 111
0.989083468914032


In [ ]:
output

(0.8486876743180412, 1)

In [ ]:
predicts = []
for i in range(len(a)):
  sample = a.iloc[i]
  id = sample["id"]
  sentence = sample["sentence"]
  pathwav = pathtowav(id)
  filesound = speech_file_to_array_fn(pathwav)
  output = rw.sound_and_sentence(filesound,sentence)
  predicts.append(output[1])


In [ ]:
a["predict"] = predicts

In [ ]:
#clean data native-speaker

In [ ]:
a

In [ ]:
a.to_csv('test_predicts.csv')

In [ ]:
import shutil

In [ ]:
shutil.move("test_predicts.csv","/content/drive/MyDrive")

In [ ]:
#end